In [1]:
import os
import fnmatch
import pandas as pd
import pickle as pk
from multiprocessing import Pool
import workers
import time

Dataset_location= "C:/Users/Admin/Desktop/Dataset/uber-pickups-in-new-york-city"

In [2]:
def get_file_location(folder_location):
    file_list= []
    for filename in os.listdir(folder_location):
        if fnmatch.fnmatch(filename, 'uber-raw-data-*14.csv'):
            full_path= os.path.join(folder_location,filename)
            file_list.append(full_path)
    return(file_list)

In [3]:
# Create a workers.py module in the same folder with the following code
# def read_csv_to_datetime(file):
#     monthly_data= pd.read_csv(file,delimiter=",",header=0)
#     monthly_data["Date/Time"]=pd.to_datetime(monthly_data["Date/Time"])
#     return monthly_data
# The purpose of this is to run the multiprocessing read and merge csv file

In [ ]:
if __name__=="__main__":
    
    print("Start")
    start= time.time()
    list_of_files= get_file_location(Dataset_location)
    pool= Pool(processes=4)
    df= pool.map(workers.read_csv_to_datetime,list_of_files)
    end= time.time()
    print("End")
    print(end-start)
    

Start


In [ ]:
new_df= pd.DataFrame()
for data in df:
    new_df= pd.concat([data, new_df])

In [ ]:
df= new_df
df.info()

In [ ]:
def get_dom(dt):
    return dt.day

def get_weekday(dt):
    return dt.weekday()

def get_hour(dt):
    return dt.hour

def get_month(dt):
    return dt.month


df['Day_of_month']= df["Date/Time"].map(get_dom)

df['weekday']= df["Date/Time"].map(get_weekday)

df['Hour']= df["Date/Time"].map(get_hour)

df["Month"]= df["Date/Time"].map(get_month)


In [ ]:
df['Month'].value_counts()

In [25]:
%pylab inline
import seaborn as sns
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [33]:
def count_rows(rows):
    return len(rows)

by_date = df.groupby(["Month","Day_of_month"]).apply(count_rows)
by_date

Month  Day_of_month
4      1               14546
       2               17474
       3               20701
       4               26714
       5               19521
       6               13445
       7               19550
       8               16188
       9               16843
       10              20041
       11              20420
       12              18170
       13              12112
       14              12674
       15              20641
       16              17717
       17              20973
       18              18074
       19              14602
       20              11017
       21              13162
       22              16975
       23              20346
       24              23352
       25              25095
       26              24925
       27              14677
       28              15475
       29              22835
       30              36251
dtype: int64